In [15]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
import dlc_pratical_prologue as prologue
from loading_datas import  generate_pair_sets
import matplotlib.pyplot as plt

## Data set


In [ ]:
N = 1000
train_input, train_target, train_classes, test_input, test_target, test_classes = generate_pair_sets(N)

In [4]:
# Double checking
print("Training Set")
print(train_input.size())
print(train_target.size())
print(train_classes.size())
print("-------------------")
print("Testing Set")
print(test_input.size())
print(test_target.size())
print(test_classes.size())

Training Set
torch.Size([1000, 2, 14, 14])
torch.Size([1000])
torch.Size([1000, 2])
-------------------
Testing Set
torch.Size([1000, 2, 14, 14])
torch.Size([1000])
torch.Size([1000, 2])


## Helping functions

In [53]:
"""
Input :-

model : Pytorch NN model
input_data : Tensor of N X 2 X 14 X 14
target_data : Tensor of N X 1
batch_size : Size of the batch

-----------------------------------

Output :-

The number of samples not well classified
"""
def compute_nb_errors(model, data_input, data_target,batch_size):

    nb_data_errors = 0

    for inputs, targets in zip(data_input.split(batch_size), data_target.split(batch_size)):
        output = model(inputs)
        _, predicted_classes = torch.max(output, 1)
        for k in range(len(targets)):
            if targets[k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1
                
    return nb_data_errors

"""
Input :-

model : Pytorch NN model
train_input : Tensor of N x 2 x 14 x 14
train_target : Tensor of N x 1
train_classes : Tensor of N x 2

test_input : Tensor of N x 2 x 14 x 14
test_target : Tensor of N x 1
test_classes : Tensor of N x 2

epochs : the number of passes of the entire training dataset
eta : learning parameter

batch_size : Size of the batch

-----------------------------------

Output :-

The number of samples not well classified

"""
eta = 0.01
def training_model(model,
                   train_input,train_target,train_classes,
                   test_input,test_target,test_classes,
                   eta,epochs = 25,
                   batch_size = 50):
  
  train_acc = []
  test_acc = []

  # define criterion and optimizer
  # need to check the other possibilities
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr = eta)

  for e in range(0,epochs):

    for inputs,target in zip(train_input.split(batch_size),train_target.split(batch_size)):

      outputs = model(inputs)
      loss = criterion(outputs, target)

      # optimising parameters
      optimizer.zero_grad()
      loss.backward()
      optimizer.step() 

  

    train_errors = 0
    train_errors = 100 * (1 - compute_nb_errors(model, train_input, train_target,batch_size)/train_input.size(0))
    test_errors = 0
    test_errors = 100 * (1 - compute_nb_errors(model, test_input, test_target,batch_size)/test_input.size(0))
    print(f"Epoch # {e+1} / Train accuracy (%): {train_errors:.2f} / Test accuracy (%): {test_errors:.2f}")
     


  return " "
  



## Models

In [50]:
# parameters
eta = 0.01

In [70]:

# Net without Weight sharing
class Net1(nn.Module):
  def __init__(self):
    super(Net1,self).__init__()

    self.conv11 = nn.Conv2d(1,16,3)
    self.conv12 = nn.Conv2d(16,32,3)

    self.conv21 = nn.Conv2d(1,16,3)
    self.conv22 = nn.Conv2d(16,32,3)

    self.pool = nn.MaxPool2d(kernel_size=(2,2),stride=2)

    self.fc1 = nn.Linear(64*4*4,64)
    self.fc2 = nn.Linear(64,32)
    self.fc3 = nn.Linear(32,2)

  def forward(self,x): 
    # spliting the channels
    c1 = torch.narrow(x,1,0,1)
    c2 = torch.narrow(x,1,1,1)

    # Channel 1
    c1 = F.relu(self.conv11(c1))
    c1 = self.pool(c1)
    c1 = F.relu(self.conv12(c1))
    

    # Channel 2
    c2 = F.relu(self.conv21(c2))
    c2 = self.pool(c2)
    c2 = F.relu(self.conv22(c2))
    


    output = torch.cat((c1,c2),1)
    output = output.view(-1,64*4*4)
   
    output = F.relu(self.fc1(output))
    output = F.relu(self.fc2(output))
    output = self.fc3(output)

    return output

# Net without Weight sharing
class Net2(nn.Module):
  def __init__(self):
    super(Net2,self).__init__()
    
    self.conv1 = nn.Conv2d(1,16,3)
    self.conv2 = nn.Conv2d(16,32,3)
    self.pool = nn.MaxPool2d(kernel_size=(2,2),stride=2)

    self.fc1 = nn.Linear(64*4*4,64)
    self.fc2 = nn.Linear(64,32)
    self.fc3 = nn.Linear(32,2)

  def forward(self,x): 
    # spliting the channels
    c1 = torch.narrow(x,1,0,1)
    c2 = torch.narrow(x,1,1,1)

    # Channel 1
    c1 = F.relu(self.conv1(c1))
    c1 = self.pool(c1)
    c1 = F.relu(self.conv2(c1))

    # Channel 2
    c2 = F.relu(self.conv1(c2))
    c2 = self.pool(c2)
    c2 = F.relu(self.conv2(c2))


    output = torch.cat((c1,c2),1)
    output = output.view(-1,64*4*4)
   
    output = F.relu(self.fc1(output))
    output = F.relu(self.fc2(output))
    output = self.fc3(output)

    return output



In [71]:
# Net without weight sharing
model1 = Net1()
training_model(model1,train_input,train_target,train_classes,test_input,test_target,test_classes,eta,epochs = 25,batch_size = 50)

Epoch # 1 / Train accuracy (%): 64.20 / Test accuracy (%): 59.90
Epoch # 2 / Train accuracy (%): 78.10 / Test accuracy (%): 76.60
Epoch # 3 / Train accuracy (%): 79.30 / Test accuracy (%): 79.20
Epoch # 4 / Train accuracy (%): 83.30 / Test accuracy (%): 80.20
Epoch # 5 / Train accuracy (%): 82.40 / Test accuracy (%): 81.00
Epoch # 6 / Train accuracy (%): 84.90 / Test accuracy (%): 82.60
Epoch # 7 / Train accuracy (%): 88.90 / Test accuracy (%): 84.10
Epoch # 8 / Train accuracy (%): 93.00 / Test accuracy (%): 83.60
Epoch # 9 / Train accuracy (%): 90.80 / Test accuracy (%): 81.20
Epoch # 10 / Train accuracy (%): 93.20 / Test accuracy (%): 81.70
Epoch # 11 / Train accuracy (%): 94.70 / Test accuracy (%): 83.30
Epoch # 12 / Train accuracy (%): 93.40 / Test accuracy (%): 81.80
Epoch # 13 / Train accuracy (%): 87.00 / Test accuracy (%): 79.00
Epoch # 14 / Train accuracy (%): 97.70 / Test accuracy (%): 84.00
Epoch # 15 / Train accuracy (%): 97.60 / Test accuracy (%): 84.30
Epoch # 16 / Train 

' '

In [72]:
# Net with weight sharing
model2 = Net2()
training_model(model2,train_input,train_target,train_classes,test_input,test_target,test_classes,eta,epochs = 25,batch_size = 50)

Epoch # 1 / Train accuracy (%): 72.70 / Test accuracy (%): 71.20
Epoch # 2 / Train accuracy (%): 74.60 / Test accuracy (%): 75.90
Epoch # 3 / Train accuracy (%): 84.90 / Test accuracy (%): 81.50
Epoch # 4 / Train accuracy (%): 83.30 / Test accuracy (%): 79.50
Epoch # 5 / Train accuracy (%): 88.60 / Test accuracy (%): 83.10
Epoch # 6 / Train accuracy (%): 90.00 / Test accuracy (%): 83.60
Epoch # 7 / Train accuracy (%): 92.90 / Test accuracy (%): 84.80
Epoch # 8 / Train accuracy (%): 94.40 / Test accuracy (%): 85.80
Epoch # 9 / Train accuracy (%): 95.90 / Test accuracy (%): 85.70
Epoch # 10 / Train accuracy (%): 96.10 / Test accuracy (%): 85.40
Epoch # 11 / Train accuracy (%): 97.00 / Test accuracy (%): 85.70
Epoch # 12 / Train accuracy (%): 98.10 / Test accuracy (%): 85.60
Epoch # 13 / Train accuracy (%): 98.10 / Test accuracy (%): 85.80
Epoch # 14 / Train accuracy (%): 98.60 / Test accuracy (%): 85.60
Epoch # 15 / Train accuracy (%): 98.70 / Test accuracy (%): 85.10
Epoch # 16 / Train 

' '